In [3]:
# Import necessary libraries
import ee

# Initialize the Earth Engine API
ee.Authenticate()

In [4]:
ee.Initialize()

EEException: ee.Initialize: no project found. Call with project= or see http://goo.gle/ee-auth.

In [1]:
# Define the AOI: Milan, Italy
milan_coordinates = [9.1859243, 45.4654219]  # Longitude, Latitude
AOI = ee.Geometry.Point(milan_coordinates).buffer(10000)  # Buffer by 10 km

# Select a dataset: Landsat 8 Surface Reflectance
dataset = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

# Filter the dataset by date and AOI
filtered_dataset = dataset.filterDate('2020-01-01', '2020-12-31').filterBounds(AOI)

# Select the first image from the filtered collection
image = filtered_dataset.first()

# Optionally, visualize the image (if using a Jupyter notebook or similar environment)
# Note: This requires the geemap library for visualization in notebooks, which is not part of the standard Earth Engine API
import geemap
Map = geemap.Map()
Map.center_object(AOI, 10)
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000, 'gamma': 1.5}, "RGB")
Map

# Export the image to Google Drive (set up required in the Earth Engine code editor)
task = ee.batch.Export.image.toDrive(**{
    'image': image.select(['B4', 'B3', 'B2']),
    'description': 'Milan_Satellite_Image',
    'folder': 'EarthEngineImages',
    'scale': 30,
    'region': AOI
})

# Start the export task
task.start()

# Monitor the task
print('Exporting Milan_Satellite_Image to Google Drive, Task:', task.id)


RefreshError: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Engine metadata service. Status: 404 Response:\nb''", <google_auth_httplib2._Response object at 0x7ec7f3a0caf0>)

In [5]:
# Import necessary libraries
import ee
import folium

def authenticate_ee():
    """Authenticate to the Earth Engine servers."""
    ee.Authenticate()

def initialize_ee():
    """Initialize the Earth Engine API."""
    ee.Initialize()

def get_satellite_image():
    # Define the area of interest (AOI) with the coordinates of Milan.
    milan_aoi = ee.Geometry.Rectangle([9.0402, 45.3860, 9.2780, 45.5356])

    # Select a satellite image dataset. Here, we use the Landsat 8 surface reflectance dataset.
    dataset = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

    # Filter the dataset for images intersecting with the AOI and date range.
    filtered_dataset = dataset.filterBounds(milan_aoi).filterDate('2023-01-01', '2023-12-31').sort('CLOUD_COVER').first()

    # Select the visualization parameters for the satellite image.
    vis_params = {
        'min': 0,
        'max': 3000,
        'bands': ['B4', 'B3', 'B2']  # RGB bands for Landsat 8
    }

    # Use folium to visualize the satellite image.
    map = folium.Map(location=[45.4642, 9.1900], zoom_start=10)
    map.add_ee_layer = lambda image, vis_params, name: map.add_child(folium.raster_layers.TileLayer(tiles=image.getMapId(vis_params)['tile_fetcher'].url_format, attr='Map Data &copy; Google Earth Engine', overlay=True, name=name))
    map.add_ee_layer(filtered_dataset, vis_params, 'Milan Satellite Image')

    # Display the map in a Jupyter notebook. Outside of Jupyter, you would save the map as an HTML file.
    return map

if __name__ == "__main__":
    # Authenticate and initialize the Earth Engine API.
    authenticate_ee()
    initialize_ee()

    # Retrieve and display the satellite image.
    milan_map = get_satellite_image()
    milan_map.save('milan_map.html')  # Save the map as an HTML file if not using Jupyter notebooks.
    print("Satellite image map of Milan saved as 'milan_map.html'. Open this file in a web browser to view the map.")


EEException: ee.Initialize: no project found. Call with project= or see http://goo.gle/ee-auth.